In [1]:
import tensorflow as tf
import numpy as np
from src.utilities import *
from src.hmm_base_class import _HMM
from src.hmm_natural_gradient import VB_HMM
import time

IndentationError: unexpected indent (hmm_natural_gradient.py, line 131)

In [2]:
N = 5
D = 4
K =4

class HMM:
    def __init__(self):
        pass

    def simulate(self,nSteps):
        def drawFromMultinomial(t):
            return np.where(np.random.multinomial(1,t) == 1)[0][0]

        states = np.zeros(nSteps, dtype=np.int)

        # Initial values. For sake of simplicity, there is only one initial prob vector
        states[0] = drawFromMultinomial(self.pi)

        for i in range(1,nSteps):
            states[i] = drawFromMultinomial(self.t[states[i-1],:])
        return states

hmm = HMM()
hmm.pi = np.array([0.25, 0.25, 0.25, 0.25])
hmm.t = np.array([
    [0.97, 0.03, 0.0, 0.0],
    [0.00, 0.97, 0.03, 0.],
    [0.0, 0.0, 0.97, 0.3],
    [0.3, 0.0, 0.0, 0.97]])

mu = np.array([[0.]*D,[1.]*D,[2.]*D,[3.]*D,])


distributions = []
for i in range(4):
    distributions.append(np.random.multivariate_normal(mu[i], np.eye(4), N).T)

state_seq = hmm.simulate(N)

data = np.zeros([4, N])
for i in range(4):
    data += (state_seq==i)*distributions[i]
    
# Observations should be T X D format
data = data.T

# Emission prior
mu_0 = np.zeros(D)
sigma_0 = 0.75*np.cov(data.T)
kappa_0 = 0.01
nu_0 = 7

prior_emit = [np.array([mu_0] * K), np.array([sigma_0] * K), np.array([kappa_0] * K), np.array([nu_0] * K)]

# Initial and transition matrix prior
A_0 = np.ones(K*K).reshape((K,K))
pi_0 = np.ones(K)

In [3]:
hmm = VB_HMM(data, pi_0, A_0, prior_emit)

In [ ]:
hmm.infer()

test1


In [5]:
def em_step():
    hmm._local_update()
    hmm._global_update()

    lb = hmm._lower_bound()
    converged = tf.abs(lb - hmm._elbo) < hmm._epsilon
    hmm._elbo = tf.assign(hmm._elbo, lb)
    
    return converged, lb

def apply_em():
    
    converged = tf.cast(False, tf.bool)
    lower_bound = tf.constant([0], dtype=tf.float32)

    for i in range(hmm._max_iter):
        with tf.name_scope('EM_step-%s' %i):
            converged, lower_bound = em_step()
    
    return converged, lower_bound

conv, lb = apply_em()

In [10]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sess.run([hmm._ln_alpha, hmm._ln_beta, hmm._ln_scale , conv, lb])

[array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]], dtype=float32), array([[ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.]], dtype=float32), array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.],
        [ 0.]], dtype=float32), True, -497.27838]

In [7]:
lb.eval()

-497.27838

In [9]:
hmm._ln_alpha.eval()

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]], dtype=float32)

TypeError: 'Tensor' object does not support item assignment